TODO: Introduction

In [1]:
from device import select_device, with_cpu, with_gpu
import json
from parse import get_config
import torch

In [2]:
config_name = 'tmp2'

In [3]:
config_path = f'configs/{config_name}.json'

# Model

In [4]:
use_gpu, device, device_idx = select_device()
config = get_config(use_gpu, device, device_idx, load_weights=True, config_path=config_path, parse_args=False)

# BLEU

# Attention

# Learning Curves